In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as spio
import getpass
import os
import csv

In [188]:
# Number of days in total
numdays = 6

# Name of the computer
myhost = getpass.getuser()

# Base folder
datafolder = 'C:/Users/username/Documents/Fieldwork/fieldwork-2019-adcp-adv/data/'.replace('username', myhost)# basefolder
basefolder = datafolder+'StreamPro'

# Subfolders
daynames = [i for i in os.listdir(basefolder)]






In [254]:
# Count the number of days
day = 0

# which files to include
includes =['*streampro_datatotal_ASC.TXT']

# loop over all roots
for root, dirs, files in os.walk(basefolder, topdown=True):
  
    # Don't look in the basefolder
    if root == basefolder:
        continue
        
    # which folder are we doing right now?
    print(daynames[day])
    
    goodfiles =[file for file in files if file[-13:] == 'total_ASC.TXT']
    
 
    
    # Loop over the correct files in the folder
    for file in goodfiles:
        # Load direction from file in folder
        print(file)
        data = pd.read_csv(root+'/'+file, delimiter = ';')
        #print(data)
        
        water_vel = pd.DataFrame.as_matrix(data['northV']) #only the velocity to the north is in direction of the river
        #print(water_vel[0].split(','))
        
        depth_bin = pd.DataFrame.as_matrix(data['depthbin'])
        
        total_width = np.array(data['totalwidth'])
        print(total_width)
        
        bin_size = data['binsize']
        
        TopQ = data['topQ']
        print(TopQ)
        
        MiddleQ = data['middleQ']
        
        LeftQ = data['LeftQ']
        
        RightQ = data['RightQ']
        

    day +=1

Monday afternoon
group 5_0_000_streampro_datatotal_ASC.TXT
[ 0.82        0.8363275   0.83065329  0.82849473  0.82750422  0.82272586
  0.8317491   0.82699266  0.835067    0.84461998  0.84461998  0.88052342
  0.88261954  0.88592557  0.88925264  0.88301159  0.88772929  0.88325866
  0.88325866  0.87188836  0.84604116  0.83010284  0.82805079  0.82939282
  0.835512    0.84326164  0.83548305  0.82433586  0.82259848  0.83717826
  0.87600061  0.88102582  0.87732602  0.89230624  0.90157887  0.89451095
  0.89418254  0.90378831  0.91734383  0.92720969  0.92704723  0.92962126
  0.94084887  0.95864956  0.9745402   0.95774011  0.94228277  0.95906963
  0.96842171  0.96676855  0.96857012  0.97427521  0.985559    0.99903891
  1.00789062  0.99959922  0.99790587  0.99608985  1.00021922  1.00021922
  0.99271829  0.98199606  0.97562429  0.97253149  0.96891335  0.96891335
  0.92989248  0.94112439  0.94940843  0.95123914  0.94647285  0.95718001
  0.95596811  0.94022774  0.92651174  0.92753974  0.91593644  0.9

C:\Users\Sanne de Smet\Documents\Python\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Sanne de Smet\Documents\Python\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Tuesday afternoon
Group 7_0_000_19-05-14_135728_streampro_datatotal_ASC.TXT
[ 1.42        1.47551019  1.48716545  1.48716545  1.48716545  1.43657048
  1.43543912  1.44170156  1.44170156  1.44170156  1.45784788  1.45784788
  1.45784788  1.49685864  1.49685864  1.52076517  1.54453317  1.54257331
  1.66322177  1.68002188  1.68215981  1.69545242  1.68880143  1.69705248
  1.69705248  1.71577497  1.71577497  1.65050436  1.6342737   1.62752568
  1.66266421  1.65791173  1.69564371  1.70827443  1.76166319  1.76851997
  1.7382206   1.83913797  1.87082143  1.87323514  1.87911971  1.86882056
  1.87606902  1.89202274  1.8839446   1.89899108  1.84179679  1.8269364
  1.85568138  1.86149159  1.85829655  1.8598921   1.8687062   1.86782647
  1.86868685  1.86868685  1.86868685  1.89476617  1.88449258  1.88215345
  1.89198448  1.8839815   1.86200765  1.86200765  1.86200765  1.86200765
  1.86200765  1.86200765  1.86200765  1.86200765  1.86200765  1.86200765
  1.86200765  1.86200765  1.86200765  1.86200765 

0      0.000000
1     -0.021740
2     -0.009647
3     -0.021421
4     -0.005529
5      0.024640
6      0.042567
7      0.079908
8      0.090411
9      0.096547
10     0.102525
11     0.114886
12     0.125415
13     0.149642
14     0.161737
15     0.176660
16     0.185162
17     0.200571
18     0.219783
19     0.233262
20     0.224860
21     0.233891
22     0.242861
23     0.257184
24     0.264346
25     0.265307
26     0.273056
27     0.277185
28     0.295725
29     0.311344
         ...   
112    1.986733
113    2.010797
114    2.017250
115    2.046425
116    2.072645
117    2.081564
118    2.099293
119    2.128700
120    2.150034
121    2.175223
122    2.191272
123    2.213838
124    2.231164
125    2.226099
126    2.253508
127    2.287042
128    2.301007
129    2.293011
130    2.298641
131    2.297474
132    2.350204
133    2.356329
134    2.390251
135    2.390251
136    2.495279
137    2.495279
138    2.495279
139    2.495279
140    2.495279
141    2.495279
Name: topQ, Length: 142,